# データ解析サンプル1

## 1. データソースについて

「令和５年における労働災害発生状況（確定）(23-16.xlsx）をもとに、データ解析を行う。

ここでは、同Excelファイルの「都道府県、業種別死亡災害発生状況（令和５年）」のデータを対象する。

## 2. 前処理

### 前処理の必要性

同Excelファイルは複数のシートに異なる対象のデータが保存され、さらに読者がそのまま読む上で分かりやすく工夫がされている。代表的なものを以下に示す。

* 表紙ページがある
* タイトル文字列や、データを取得した日など、表そのものの齟齬側の情報が記載されている
* １つのシートに２つの表がある
* 表の中に小計がある

これらの表データ以外の情報は自動で認識することができないため、あらかじめ処理しやすい形式に自動、手動で変換しておく必要がある。これを前処理と呼ぶ。

### どのような前処理をしたか

前処理の内容を以下に示す。

1. 元のExcelファイルのシート「都道府県、業種別死亡災害発生状況（令和５年）」を取り出す
2. 不要な行、列を削除
3. 小計、合計の欄を削除
4. 小分類（製造、鉱業、建設、運輸、貨物、農林、他）ごとにシートを分割

## 3. Excelファイルの読み込み

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
EXCEL_FILE = 'data/都道府県、業種別死亡災害発生状況（令和５年）.xlsx'
dfs = pd.read_excel(EXCEL_FILE, sheet_name=None, index_col=0)
dfs.keys()

## 4. 製造業のデータを分析する

### 4-1. 製造業のDataFrameを取り出す

In [ ]:
df_prod = dfs['製造']
df_prod

DataFrameのカラムごとの要約統計量を表示するメソッド、describe()を呼び出してみる。<br>
表示されるものは以下の通り。

* **count**: 欠損値NaNではない要素の個数
* **mean**: 算術平均
* **std**: 標準偏差
* **min**: 最小値
* **25%**: 1/4分位数
* **50%**: 中央値（median）
* **75%**: /4分位数
* **max**: 最大値

参考：**pandas.DataFrame.describe**<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html

In [ ]:
df_prod.describe()

### 4-2. 小分類ごと死亡災害発生数を表示

* `df.sum()`で合計値を計算
* `axis=0`でインデックス集計を指定
* `Series.sort_values()`でソート
  * `ascending=True`で昇順（デフォルト）、`False`で降順

In [ ]:
ser = df_prod.sum(axis=0).sort_values(ascending=False)
ser.plot.bar()
plt.show()

### 4-3. 食料関係の県別死亡事故発生数のヒストグラム

In [ ]:
plt.hist(df_prod['食料'])
plt.show()

食料関係の製造業での死亡事故、大体の県は0件、あっても1件、稀に2,3件。

1件以上の都道府県が気になるので、表示してみる。

In [ ]:
ser = df_prod[(df_prod['食料'] >= 1)]['食料'].sort_values(ascending=False)
ser.plot.bar()
plt.show()

北海道、鹿児島が最多で3件ということがわかる。食品加工業が盛んということか。

### 4-4. 都道府県別の製造業死亡災害発生数を表示

* `df.sum()`で合計値を計算
* `axis=1`でカラム集計を指定

県別の件数でヒストグラムを表示する、

In [ ]:
ser = df_prod.sum(axis=1)
plt.hist(ser)
plt.show()

概ね0〜6件だが、8, 9件の死亡事故が発生している県がある。

どこの県か調べる。

In [ ]:
ser = df_prod.sum(axis=1).sort_values(ascending=False).head(10)
ser.plot.bar()
plt.show()

最多だった福岡県の、製造業における死亡事故数を調べる。

In [ ]:
ser = df_prod.loc['福岡'].sort_values(ascending=False)
ser = ser[ser > 0]
ser.plot.bar()
plt.show()

## 5. 全体を集計する

まずDataFrame（製造業の死亡災害発生数）の合計値を出す。

In [ ]:
# 製造業の死亡災害発生数
print(df_prod.sum().sum())   # axis=0は省略
print(df_prod.values.sum())  # こういう書き方もある

全シート（全分類）の合計値を出す。

In [ ]:
# 死亡災害発生数の合計
total = np.int64(0)  # 単に0を代入してもいい
for key, df in dfs.items():
    sub_total = df.sum().sum()
    print(key, sub_total)  # 分類ごとの小計を表示
    total += sub_total

print('合計', total)

上記のforループは、以下のように内包表記にすることができる。

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

# 内包表記を使う
ser = pd.Series({key: int(df.sum().sum()) for key, df in dfs.items()})
ser = ser.sort_values(ascending=False)
print(ser)
print(f'合計 {ser.sum()}')

# ついでにグラフ化
ser.plot.bar()
plt.show()

この中で建設業が多いのが気になる。

小分類で内訳を見る。

In [ ]:
df_const = dfs['建設']
ser = df_const.sum(axis=0).sort_values(ascending=False)
ser.plot.bar()
plt.show()

どこの都道府県が多い？

In [ ]:
ser = df_const.sum(axis=1).sort_values(ascending=False).head(10)
ser.plot.bar()
plt.show()